In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import seaborn as sns

In [2]:
# import labels for gudhi shapes
gudhi_shape_labels = np.genfromtxt('Gudhi Shape Dataset/shape_labels.csv', delimiter=',', skip_header=1)
gudhi_shape_labels = gudhi_shape_labels.astype(int)[:,2]
print(len(gudhi_shape_labels))

2000


In [3]:
seed_value = 221
np.random.seed(seed_value)

num_samples = 2000 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(gudhi_shape_labels), size=num_samples, replace=False)
base = 'Gudhi Shape Dataset/'
# Select the corresponding data and labels
gudhi_laplacians = []
gudhi_vr_persistence_images = []
gudhi_abstract_persistence_images = []
gudhi_selected_labels = []

for i in random_indices:
    gudhi_laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    gudhi_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_selected_labels.append(gudhi_shape_labels[i])

# Convert selected labels to NumPy array
gudhi_selected_labels = np.array(gudhi_selected_labels)

# Print summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(gudhi_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(gudhi_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(gudhi_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {gudhi_selected_labels.shape}")

Randomly selected 2000 samples.
Shape of laplacians: (2000, 1000, 1000)
Shape of VR persistence images: (2000, 100, 100)
Shape of abstract persistence images: (2000, 100, 100)
Shape of selected labels: (2000,)


In [4]:
# import labels for medical shapes
medical_shape_labels = np.genfromtxt('Medical Dataset/shape_labels.csv', delimiter=',', skip_header=1)
medical_shape_labels = medical_shape_labels.astype(int)[:,2]
print(len(medical_shape_labels))

162


In [5]:
seed_value = 221
np.random.seed(seed_value)

num_samples = 162 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(medical_shape_labels), size=num_samples, replace=False)
base = 'Medical Dataset/'
# Select the corresponding data and labels
medical_laplacians = []
medical_vr_persistence_images = []
medical_abstract_persistence_images = []
medical_selected_labels = []

for i in random_indices:
    medical_laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels = np.array(medical_selected_labels)

# Print summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {medical_selected_labels.shape}")

Randomly selected 162 samples.
Shape of laplacians: (162, 1000, 1000)
Shape of VR persistence images: (162, 100, 100)
Shape of abstract persistence images: (162, 100, 100)
Shape of selected labels: (162,)


In [6]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [7]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

# Split Gudhi Laplacians
train_data_gudhi_laplacians, test_data_gudhi_laplacians, train_labels_gudhi, test_labels_gudhi = train_test_split(
    gudhi_laplacians, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_laplacians, test_data_gudhi_laplacians, valid_labels_gudhi, test_labels_gudhi = train_test_split(
    test_data_gudhi_laplacians, test_labels_gudhi, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Laplacians (valid/test split only, since all are used for validation/testing)
valid_data_medical_laplacians, test_data_medical_laplacians, valid_labels_medical, test_labels_medical = train_test_split(
    medical_laplacians, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Laplacians and labels
valid_laplacians = valid_data_gudhi_laplacians + valid_data_medical_laplacians
valid_labels = np.concatenate((valid_labels_gudhi, valid_labels_medical))

# Combine test sets for completeness
test_laplacians = test_data_medical_laplacians
test_labels = test_labels_medical

# Print a summary
print(f"Laplacians Train data size: {len(train_data_gudhi_laplacians)}")
print(f"Laplacians Validation data size: {len(valid_laplacians)}")
print(f"Laplacians Test data size: {len(test_laplacians)}")


# Split VR Persistence Images
train_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_vr_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_vr_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical VR Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_vr_persistence_images, test_data_medical_vr_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_vr_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for VR Persistence Images and labels
valid_vr_persistence_images = valid_data_gudhi_vr_persistence_images + valid_data_medical_vr_persistence_images
valid_vr_labels = np.concatenate((valid_labels_gudhi, valid_labels_medical))

# Combine test sets for VR Persistence Images and labels
test_vr_persistence_images = test_data_medical_vr_persistence_images
test_vr_labels = test_labels_medical


# Split Abstract Persistence Images
train_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_abstract_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_abstract_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Abstract Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_abstract_persistence_images, test_data_medical_abstract_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_abstract_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Abstract Persistence Images and labels
valid_abstract_persistence_images = valid_data_gudhi_abstract_persistence_images + valid_data_medical_abstract_persistence_images
valid_abstract_labels = np.concatenate((valid_labels_gudhi, valid_labels_medical))

# Combine test sets for Abstract Persistence Images and labels
test_abstract_persistence_images = test_data_medical_abstract_persistence_images
test_abstract_labels = test_labels_medical


Laplacians Train data size: 1600
Laplacians Validation data size: 281
Laplacians Test data size: 81


In [8]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes=2):
        super(CNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling to resize to 100x100
        self.adaptive_pool = nn.AdaptiveAvgPool2d((100, 100))
        
        # Dynamically calculate input size to fc1
        self.feature_size = self._get_feature_size(input_shape)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_feature_size(self, input_shape):
        # Create a dummy input to calculate size after conv and pooling
        dummy_input = torch.zeros(1, 1, *input_shape)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to get 100x100 size
        x = self.adaptive_pool(x)
        return x.numel()  # Number of elements after flattening

    def forward(self, x):
        # Apply convolutional layers with pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to resize to 100x100
        x = self.adaptive_pool(x)
        
        # Flatten and pass through fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

class DualInputCNN(nn.Module):
    def __init__(self, input_shape1, input_shape2, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Laplacian input path with additional pooling to reduce to 100x100
        self.conv1_lap = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_lap = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool_lap = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions
        self.adaptive_pool_lap = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Persistence image input path (no pooling)
        self.conv1_pers = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_pers = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool_pers = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 100 * 100 + 32 * 100 * 100, 128)  # Adjusted for 100x100 input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x1, x2):
        # Laplacians path (downsampling to 100x100)
        x1 = F.relu(self.conv1_lap(x1))
        x1 = self.pool_lap(x1)  # First pool: 250x250 -> 125x125
        x1 = F.relu(self.conv2_lap(x1))
        x1 = self.pool_lap(x1)  # Second pool: 125x125 -> 62x62
        x1 = self.adaptive_pool_lap(x1)  # Resize to 100x100
        
        # Persistence images path (no pooling)
        x2 = F.relu(self.conv1_pers(x2))
        x2 = F.relu(self.conv2_pers(x2))
        x2 = self.adaptive_pool_pers(x2)  # Ensure persistence images are 100x100
        
        # Concatenate along dim=1 (channels)
        x = torch.cat((x1, x2), dim=1)  # Concatenates the outputs along the channel axis

        # Flatten for fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


In [9]:
# Convert data to PyTorch Datasets
test_dataset_laplacians = ShapeDataset(test_laplacians, test_labels)
test_dataset_vr = ShapeDataset(test_vr_persistence_images, test_vr_labels)
test_dataset_abstract = ShapeDataset(test_abstract_persistence_images, test_abstract_labels)

# Define DataLoaders
batch_size = 32

test_loader_laplacians = torch.utils.data.DataLoader(test_dataset_laplacians, batch_size=batch_size, shuffle=False)
test_loader_vr = torch.utils.data.DataLoader(test_dataset_vr, batch_size=batch_size, shuffle=False)
test_loader_abstract = torch.utils.data.DataLoader(test_dataset_abstract, batch_size=batch_size, shuffle=False)

In [11]:
# For single-input CNN (Laplacians)
input_shape = train_data_gudhi_laplacians[0].shape
num_classes = 2  # binary classification
num_epochs = 10

# Create and manage single-input CNN models
models_single_laplacians = {}
for epoch in range(1, num_epochs + 1):
    model_name = f"Trained CNNs/model_single_laplacians_epoch_{epoch}"
    model = CNN(input_shape, num_classes)
    model.load_state_dict(torch.load(f"{model_name}.pth"))
    model.eval()
    models_single_laplacians[epoch] = model


# For dual-input CNNs (Laplacians + VR Persistence Images, Laplacians + Abstract Persistence Images)
input_shape1 = train_data_gudhi_laplacians[0].shape
input_shape2 = train_data_gudhi_vr_persistence_images[0].shape
input_shape3 = train_data_gudhi_abstract_persistence_images[0].shape

# Manage dual-input CNNs for VR and Abstract datasets
models_dual_lap_vr = {}
models_dual_lap_abstract = {}

for epoch in range(1, num_epochs + 1):
    # Laplacians + VR
    model_name_vr = f"Trained CNNs/model_dual_lap_vr_epoch_{epoch}"
    model_vr = DualInputCNN(input_shape1, input_shape2, num_classes)
    model_vr.load_state_dict(torch.load(f"{model_name_vr}.pth"))
    model_vr.eval()
    models_dual_lap_vr[epoch] = model_vr

    # Laplacians + Abstract
    model_name_abstract = f"Trained CNNs/model_dual_lap_abstract_epoch_{epoch}"
    model_abstract = DualInputCNN(input_shape1, input_shape3, num_classes)
    model_abstract.load_state_dict(torch.load(f"{model_name_abstract}.pth"))
    model_abstract.eval()
    models_dual_lap_abstract[epoch] = model_abstract


/var/folders/d3/99lvl9fd1673ngz9966pvq100000gn/T/ipykernel_6943/705805221.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_name

In [12]:
def test_single_input(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(dataloader, desc="Testing", leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm


def test_dual_input(model, dataloader1, dataloader2, criterion, device='cuda'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(zip(dataloader1, dataloader2), desc="Testing (Dual Input)", leave=False, total=min(len(dataloader1), len(dataloader2)))

    with torch.no_grad():
        for (inputs1, _), (inputs2, targets) in progress_bar:
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)

            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, targets)
            running_loss += loss.item()

            if outputs.shape[-1] > 1:
                _, predicted = torch.max(outputs, 1)
            else:  # Binary classification
                predicted = (outputs > 0.5).float()

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = running_loss / len(dataloader1.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
criterion = nn.CrossEntropyLoss()

num_epochs = 10

In [20]:
# Initialize empty lists to store all metrics for each epoch
test_metrics_single_list = []
test_metrics_dual_vr_list = []
test_metrics_dual_abstract_list = []

# Initialize empty lists to store all confusion matrices for each epoch
cm_single_list = []
cm_dual_vr_list = []
cm_dual_abstract_list = []

for epoch in range(1, num_epochs + 1):  # Start from 1 since epochs in the dictionaries are 1-indexed
    print(f"Epoch {epoch} / {num_epochs}")

    # Access the models for the current epoch
    print("Testing Single Input Model (Laplacians)")
    model_single = models_single_laplacians[epoch]
    test_metrics_single, cm_single = test_single_input(model_single, test_loader_laplacians, criterion, device)
    test_metrics_single_list.append(test_metrics_single)
    cm_single_list.append(cm_single)

    print("Testing Dual Input Model (Laplacians + VR Persistence Images)")
    model_dual_vr = models_dual_lap_vr[epoch]
    test_metrics_dual_vr, cm_dual_vr = test_dual_input(
        model_dual_vr, test_loader_laplacians, test_loader_vr, criterion, device
    )
    test_metrics_dual_vr_list.append(test_metrics_dual_vr)
    cm_dual_vr_list.append(cm_dual_vr)

    print("Testing Dual Input Model (Laplacians + Abstract Persistence Images)")
    model_dual_abstract = models_dual_lap_abstract[epoch]
    test_metrics_dual_abstract, cm_dual_abstract = test_dual_input(
        model_dual_abstract, test_loader_laplacians, test_loader_abstract, criterion, device
    )
    test_metrics_dual_abstract_list.append(test_metrics_dual_abstract)
    cm_dual_abstract_list.append(cm_dual_abstract)

Epoch 1 / 10
Testing Single Input Model (Laplacians)


Testing:   0%|          | 0/3 [00:00<?, ?it/s]

Test Loss: 0.1669, Test Accuracy: 0.6420, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Testing Dual Input Model (Laplacians + VR Persistence Images)


/opt/homebrew/anaconda3/envs/FiltrationsForGNNsLargeData/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing (Dual Input):   0%|          | 0/3 [00:00<?, ?it/s]

...

Testing (Dual Input):   0%|          | 0/3 [00:00<?, ?it/s]

Test Loss: 0.1736, Test Accuracy: 0.3704, Precision: 0.3472, Recall: 0.8621, F1 Score: 0.4950
Testing Dual Input Model (Laplacians + Abstract Persistence Images)


Testing (Dual Input):   0%|          | 0/3 [00:00<?, ?it/s]

Test Loss: 0.3801, Test Accuracy: 0.6420, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


/opt/homebrew/anaconda3/envs/FiltrationsForGNNsLargeData/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Convert the lists of metrics into DataFrames with epoch as the index
test_metrics_single_df = pd.concat(test_metrics_single_list, ignore_index=True)
test_metrics_dual_vr_df = pd.concat(test_metrics_dual_vr_list, ignore_index=True)
test_metrics_dual_abstract_df = pd.concat(test_metrics_dual_abstract_list, ignore_index=True)

# Add 'Epoch' as new column
epochs = list(range(1, num_epochs + 1))

test_metrics_single_df['Epoch'] = epochs
test_metrics_dual_vr_df['Epoch'] = epochs
test_metrics_dual_abstract_df['Epoch'] = epochs

# Set 'Epoch' as index
test_metrics_single_df.set_index('Epoch', inplace=True)
test_metrics_dual_vr_df.set_index('Epoch', inplace=True)
test_metrics_dual_abstract_df.set_index('Epoch', inplace=True)

# Rename first two columns to 'Test Loss' and 'Test Accuracy'
test_metrics_single_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_vr_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_abstract_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)

# Print final test metrics for all models
print("\nSingle-input model (Laplacians) Test Metrics:")
print(test_metrics_single_df)

print("\nDual-input model (Laplacians + VR Persistence Images) Test Metrics:")
print(test_metrics_dual_vr_df)

print("\nDual-input model (Laplacians + Abstract Persistence Images) Test Metrics:")
print(test_metrics_dual_abstract_df)


Single-input model (Laplacians) Test Metrics:
       Test Loss  Test Accuracy  Precision  Recall  F1 Score
Epoch                                                       
1       0.166900       0.641975        0.0     0.0       0.0
2       0.349204       0.641975        0.0     0.0       0.0
3       0.393054       0.641975        0.0     0.0       0.0
4       0.434565       0.641975        0.0     0.0       0.0
5       0.418305       0.641975        0.0     0.0       0.0
6       0.548175       0.641975        0.0     0.0       0.0
7       0.489510       0.641975        0.0     0.0       0.0
8       0.550089       0.641975        0.0     0.0       0.0
9       0.578930       0.641975        0.0     0.0       0.0
10      0.683307       0.641975        0.0     0.0       0.0

Dual-input model (Laplacians + VR Persistence Images) Test Metrics:
       Test Loss  Test Accuracy  Precision    Recall  F1 Score
Epoch                                                         
1       0.103481       0.3

In [22]:
# Export test metrics to CSV files

test_metrics_single_df.to_csv('test_metrics_single_laplacian_medical.csv')
test_metrics_dual_vr_df.to_csv('test_metrics_dual_lap_vr_medical.csv')
test_metrics_dual_abstract_df.to_csv('test_metrics_dual_lap_abstract_medical.csv')

In [23]:
# Visualize and export confusion matrices to "Confusion\ Matrices/" folder

# Single-input model (Laplacians)
for i, cm in enumerate(cm_single_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Single-input Model (Laplacians), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/single_laplacians_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + VR Persistence Images)
for i, cm in enumerate(cm_dual_vr_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + VR Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_lap_vr_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + Abstract Persistence Images)
for i, cm in enumerate(cm_dual_abstract_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + Abstract Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_lap_abstract_cm_epoch_{i+1}.png")
    plt.close()